In [1]:
from utils.imports import *
from utils.start_spark import spark
! start http://localhost:4040/jobs/

In [2]:
doing_features = False

In [3]:
if doing_features:
    train = spark.read.parquet(".\\data\\df_train\\")
    test = spark.read.parquet(".\\data\\df_test\\")

In [4]:
if 'stages' in sys.modules: del sys.modules['stages']
from stages import *

pipeline_no_ML = Pipeline(stages = [
    the_transformer(
        add_id,
        (elapsed_to_diff, "elapsed_time", "elapsed_diff_ms"),
        (negative_to_0, "elapsed_diff_ms"),
        (elapsed_to_hours, "elapsed_time", "elapsed_time_h"),
        agg1,
        agg2,
        typeOfText,
    ),
    VectorAssembler(inputCols = ["inv_total_time_h", "inv_total_time_h_0-4", "inv_total_time_h_5-12"], outputCol = "inv_time_stand_per_group"),
    StandardScaler(inputCol = "inv_time_stand_per_group", outputCol = "inv_time_standed_per_group", withMean = True),
    StringIndexer(inputCol = "type_of_script", outputCol = "index_of_type_of_script"),
    OneHotEncoder(inputCol = "index_of_type_of_script", outputCol="dummies_of_type_of_script"), #sparse vector
])
if doing_features:
    transformer = pipeline_no_ML.fit(train)
    trans_train = transformer.transform(train)
    # pivoted {
    from files.dfs import train_labels as labels
    if labels._isRead == False: labels.read(spark)
    splited = labels.df \
        .select(
            split("session_id", "_").alias("both"),
            "correct"
        ).select(
            col("both")[0].alias("session_id"),
            col("both")[1].alias("question"),
            col("correct").alias("isCorrect")
        )
    pivoted = splited \
        .groupby("session_id") \
        .pivot("question") \
        .agg(first("isCorrect"))
    # }
    # add pivoted
    trans_joined = trans_train.join(pivoted, ["session_id"], "left")
    trans_joined.drop(trans_joined.session_id)
    trans_joined.write.mode("overwrite").parquet(r"data\trans_train")
    
    trans_test = transformer.transform(test)
    trans_test.drop(trans_test.session_id)
    trans_test.write.mode("overwrite").parquet(r"data\trans_test")
    
    trans_train = trans_joined

In [5]:
if not doing_features:
    trans_train = spark.read.parquet(r"data\trans_train")
    trans_test = spark.read.parquet(r"data\trans_train")

In [6]:
features_ass = VectorAssembler(inputCols = [
 'max_index',
 'obs_opcional',
 'obs_no_in',
 'notebook_opens',
 'notebook_explorer',
 'fullscreen',
 'hq',
 'music',
 'avg_elapsed_diff_ms_cutscene',
 'avg_elapsed_diff_ms_person',
 'avg_elapsed_diff_ms_navigate',
 'inv_time_standed_per_group',
 'dummies_of_type_of_script'
    ], outputCol="features")

In [7]:
training = True

In [8]:
# Regressão logística
stages = [features_ass]
maxIter = 10
regularization = 0.01
for i in range(1, 19):
    lr = LogisticRegression(
        labelCol = f"q{i}",
        predictionCol=f"q{i}_pred",
        maxIter = maxIter,
        standardization = False,
        regParam=regularization,
        rawPredictionCol = f"q{i}_pred_raw",
        probabilityCol = f"q{i}_pred_prob"
    )
    stages.append(lr)
lr_pipeline = Pipeline(stages = stages)

In [9]:
# Regressão logística treino e previsão
if training:
    fit = lr_pipeline.fit(trans_train)
    prevision_lr = fit.transform(trans_test)
    prevision_lr.write.mode("overwrite").parquet(r".\data\test_lr")

In [10]:
if not training:
    prevision_lr = spark.read.parquet(r".\data\test_lr")

In [19]:
# SVM
stages = [features_ass]
maxIter = 10
regularization = 0.01
for i in range(1, 19):
    svm = LinearSVC(
        maxIter = maxIter,
        regParam = regularization,
        labelCol = f"q{i}",
        predictionCol=f"q{i}_pred_svm",
        rawPredictionCol = f"q{i}_pred_raw",
        standardization = False
    )
    stages.append(svm)
svm_pipeline = Pipeline(stages = stages)

In [20]:
if training:
    fit = svm_pipeline.fit(trans_train)
    prevision_svm = fit.transform(trans_test)
    prevision_svm.write.mode("overwrite").parquet(r".\data\test_svm")


In [21]:
prevision_svm

session_id,max_index,inv_total_time_h_0-4,inv_total_time_h_5-12,inv_total_time_h,obs_opcional,obs_no_in,notebook_opens,notebook_explorer,fullscreen,hq,music,avg_elapsed_diff_ms_cutscene,avg_elapsed_diff_ms_person,avg_elapsed_diff_ms_navigate,type_of_script,inv_time_stand_per_group,inv_time_standed_per_group,index_of_type_of_script,dummies_of_type_of_script,q1,q10,q11,q12,q13,q14,q15,q16,q17,q18,q2,q3,q4,q5,q6,q7,q8,q9,features,q1_pred_raw,q1_pred_svm,q2_pred_raw,q2_pred_svm,q3_pred_raw,q3_pred_svm,q4_pred_raw,q4_pred_svm,q5_pred_raw,q5_pred_svm,q6_pred_raw,q6_pred_svm,q7_pred_raw,q7_pred_svm,q8_pred_raw,q8_pred_svm,q9_pred_raw,q9_pred_svm,q10_pred_raw,q10_pred_svm,q11_pred_raw,q11_pred_svm,q12_pred_raw,q12_pred_svm,q13_pred_raw,q13_pred_svm,q14_pred_raw,q14_pred_svm,q15_pred_raw,q15_pred_svm,q16_pred_raw,q16_pred_svm,q17_pred_raw,q17_pred_svm,q18_pred_raw,q18_pred_svm
20100114570801148,1421,10.02763169622961,2.6812586721960177,1.3300967645396202,6,3,14,10,0,0,1,1552.7769940787184,1496.467254801253,2123.454380188598,nohumor,"[1.3300967645396202,10.02763169622961,2.6812586...","[-0.025444138892526887,-0.11799623809054076,-0....",2.0,"(3,[2],[1.0])",0,0,0,0,0,0,1,1,1,1,1,0,1,0,1,1,1,1,"[1421.0,6.0,3.0,14.0,10.0,0.0,0.0,1.0,1552.7769...","[-0.9656669264531932,0.9656669264531932]",1.0,"[-1.0955279614686555,1.0955279614686555]",1.0,"[-1.024897165923841,1.024897165923841]",1.0,"[-1.0141609326262169,1.0141609326262169]",1.0,"[0.18219478798523525,-0.18219478798523525]",0.0,"[-0.98929852969251,0.98929852969251]",1.0,"[-1.0043303134507477,1.0043303134507477]",1.0,"[-0.9653784177147551,0.9653784177147551]",1.0,"[-1.0002187127840996,1.0002187127840996]",1.0,"[0.360847355012774,-0.360847355012774]",0.0,"[-0.9726986849589824,0.9726986849589824]",1.0,"[-1.0036278104598273,1.0036278104598273]",1.0,"[0.9807680734014901,-0.9807680734014901]",0.0,"[-0.9849570091339235,0.9849570091339235]",1.0,"[0.6403939017809077,-0.6403939017809077]",0.0,"[-0.9975819241211379,0.9975819241211379]",1.0,"[-0.9941495074079012,0.9941495074079012]",1.0,"[-1.025958957036342,1.025958957036342]",1.0
20100308381327090,1089,8.003646105448038,2.6110664248045508,1.035212166733572,4,4,3,0,0,0,1,8470.789860139861,2900.66011090905,2106.2898205765846,dry,"[1.035212166733572,8.003646105448038,2.61106642...","[-0.036634461493900365,-0.16812670211346706,-0....",1.0,"(3,[1],[1.0])",1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,0,0,1,"[1089.0,4.0,4.0,3.0,0.0,0.0,0.0,1.0,8470.789860...","[-0.9912012975801754,0.9912012975801754]",1.0,"[-1.1318665252394866,1.1318665252394866]",1.0,"[-1.0098119227643045,1.0098119227643045]",1.0,"[-1.0019751269844654,1.0019751269844654]",1.0,"[-0.5474463249596879,0.5474463249596879]",1.0,"[-1.0012264653869594,1.0012264653869594]",1.0,"[-1.001572548538747,1.001572548538747]",1.0,"[-0.9895206284161476,0.9895206284161476]",1.0,"[-0.9975257907393359,0.9975257907393359]",1.0,"[-0.22153677159749652,0.22153677159749652]",1.0,"[-0.9896130151273201,0.9896130151273201]",1.0,"[-1.0088221611838353,1.0088221611838353]",1.0,"[1.0073549772852113,-1.0073549772852113]",0.0,"[-0.9963786000325683,0.9963786000325683]",1.0,"[-0.1574343425579574,0.1574343425579574]",1.0,"[-0.9992258938166055,0.9992258938166055]",1.0,"[-0.9964242727224785,0.9964242727224785]",1.0,"[-1.035988108225579,1.035988108225579]",1.0
20100317593525716,1622,29.82576780639763,8.329997477861875,2.533733850965775,3,0,0,0,0,0,1,669.9261695906433,455.83151745495496,920.3795995503655,normal,"[2.533733850965775,29.82576780639763,8.32999747...","[0.020231652221340723,0.37236780131377173,0.086...",0.0,"(3,[0],[1.0])",1,0,1,1,0,0,0,1,1,1,1,1,0,0,1,1,1,0,"(17,[0,1,7,8,9,10,11,12,13,14],[1622.0,3.0,1.0,...","[-0.9596081142613693,0.9596081142613693]",1.0,"[-1.0428739996835585,1.0428739996835585]",1.0,"[-1.0120801858433994,1.0120801858433994]",1.0,"[-0.9869928240592667,0.9869928240592667]",1.0,"[0.46060582985658094,-0.46060582985658094]",0.0,"[-0.9935560845155896,0.9935560845155896]",1.0,"[-0.9939954013040896,0.9939954013040896]",1.0,